# Acquisition and Analysis

Write scripts in your favourite language to perform the following tasks:


## Task -xml

**Dutch(нідерландська)**

файл формат .xml.bz2

NL -  articles, templates, media/file descriptions, and primary meta-pages 

**Структура**
`<page>
 <title>наше_слово</title>
 <text> 
     {{=nld=}} - ідентифікатор нідерландської мови
     {{-syn-}} - ідентифікатор синонімів
    *[[наші_синоніми]]`
    
regexp шукаємо ідентифікатори та використовуємо прапорці boolean типу, що нижче шукаємо рядок з синонімами

**Вихід**

слово - синоніми

NL -  articles, templates, media/file descriptions, and primary meta-pages 

In [78]:
pd.set_option('display.max_colwidth', 400)

In [2]:
import xml.sax
import re
import pandas as pd

In [3]:
# create an XMLReader
parser = xml.sax.make_parser()

In [72]:
lst = []
class wikiHandler( xml.sax.ContentHandler ):
    def __init__(self):
        self.page = ""
        self.title = ""
        self.text = ""
        self.flag = False
        self.flag2 = False

    # Call when an element starts
    def startElement(self, tag, attributes):
        self.CurrentData = tag

    # Call when an elements ends
    def endElement(self, tag):
        self.flag = False
        self.flag2 = False
        self.CurrentData = ""   

    # Call when a character is read
    def characters(self, content):
        if self.CurrentData == "title":
            self.title = content
            
        elif self.CurrentData == "text":  
            if self.flag2:
                if  content !='\n':
                    re_synonym = re.compile('\[\[([a-zA-Z]*)]]')
                    synonyms = re_synonym.findall(content)
                    if synonyms != []:
                        lst.append([self.title, synonyms]) 
#                         print(self.title, "!!! ", synonyms)
                    self.flag2 = False
                    self.flag = False
                self.text = content    
            if re.match(r'.*{{=nld=}}.*', content):
                self.flag = True
            if self.flag and re.match(r'.*{{-syn-}}.*', content): 
                self.flag2 = True
                    

In [73]:
Handler = wikiHandler()
# A large file
xmlfile = "nlwiktionary-20190301-pages-articles-multistream.xml"
parser.setContentHandler( Handler)
xml.sax.parse(xmlfile, Handler)

In [67]:
df = pd.DataFrame(lst, columns=['word', 'synonyms'])

In [75]:
print( '{} word-synonyms pairs are found '.format(len(df)))

26501 word-synonyms pairs are found 


In [80]:
df.head(40)

,word,synonyms
0,ja,[oder]
1,de,[e]
2,één,[een]
3,goed,"[geschikt, bruikbaar, behoorlijk, correct, foutloos, juist, klasse, prima, mooi, ruim, akkoord]"
4,zelfstandig naamwoord,"[nomen, substantief, substantivum]"
5,bijvoeglijk naamwoord,"[adjectief, adjectivum]"
6,januari,[louwmaand]
7,februari,[sprokkelmaand]
8,maart,[maarte]
9,april,[lentemaand]


In [70]:
df.to_csv(path_or_buf='nl_synonyms.csv', sep=',',  header=True, index=False)

## Task - web scraping

## Task - dbpedia 

In [3]:
pd.set_option('display.max_colwidth', 400)

### завдання: Витягнемо людей української національності
використаю витягнення даних з запиту

`PREFIX dbo: <http://dbpedia.org/ontology/> 
PREFIX dbp: <http://dbpedia.org/property/> 
PREFIX xsd:     <http://www.w3.org/2001/XMLSchema#> 
select ?person ?nationality where { 
?person a dbo:Person . 
?person dbp:nationality ?nationality . 
FILTER (xsd:string(?nationality) = "Ukraine") 
} LIMIT 10 `


In [4]:
import requests
query = 'https://dbpedia.org/sparql?default-graph-uri=http%3A%2F%2Fdbpedia.org&query=PREFIX+dbo%3A+%3Chttp%3A%2F%2Fdbpedia.org%2Fontology%2F%3E+%0D%0APREFIX+dbp%3A+%3Chttp%3A%2F%2Fdbpedia.org%2Fproperty%2F%3E+%0D%0APREFIX+xsd%3A+++++%3Chttp%3A%2F%2Fwww.w3.org%2F2001%2FXMLSchema%23%3E+%0D%0Aselect+%3Fperson+%3Fnationality+where+%7B+%0D%0A%3Fperson+a+dbo%3APerson+.+%0D%0A%3Fperson+dbp%3Anationality+%3Fnationality+.+%0D%0AFILTER+%28xsd%3Astring%28%3Fnationality%29+%3D+%22Ukraine%22%29+%0D%0A%7D+LIMIT+10%0D%0A&format=application%2Fsparql-results%2Bjson&CXML_redir_for_subjs=121&CXML_redir_for_hrefs=&timeout=30000&debug=on&run=+Run+Query+'
req = requests.get(query)
data = req.json()
for result in data["results"]["bindings"]:
    print(result["person"]["value"])

http://dbpedia.org/resource/Roman_Romanchuk_(boxer)
http://dbpedia.org/resource/Dimitrij_Prokopcov
http://dbpedia.org/resource/Valentin_Mankin
http://dbpedia.org/resource/Taras_Pastukh
http://dbpedia.org/resource/Kateryna_Zubkova
http://dbpedia.org/resource/Maryna_Tkachenko
http://dbpedia.org/resource/Iurii_Tsaruk
http://dbpedia.org/resource/Yurii_Andrukhovych
http://dbpedia.org/resource/Olga_Bielkova
http://dbpedia.org/resource/Grigory_Gamarnik


### завдання: Витягнемо архітекторів американської національності, народжених після 2000 року (з описом)
використаю витягнення даних з SPARQLWrapper


In [5]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
import json

sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery("""
    PREFIX dbo: <http://dbpedia.org/ontology/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    select ?person ?abstract
    where {
    ?person a dbo:Person .
    ?person rdf:type dbo:Architect .
    ?person dbp:nationality ?nationality .
    FILTER (xsd:string(?nationality) = "USA") 
    ?person dbo:abstract ?abstract .
    ?person dbo:birthDate ?birthDate .
    FILTER ( ?birthDate > xsd:date("2000-01-01") ) 
}
LIMIT 10
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

data = []
for result in results["results"]["bindings"]:
    data.append([result["person"]["value"], result["abstract"]["value"]])

pd.DataFrame(data, columns=['link', 'abstract'])

,link,abstract
0,http://dbpedia.org/resource/August_Geiger_(architect),"August Geiger (September 2, 1887 - 1968) was one of the most prominent American architects in South Florida from 1905 to the late 1940s. He experimented in Mission, Neo-Renaissance and Art Deco architecture, but is most noted for his works in the Mediterranean Revival style. A number of his works are listed on the U.S. National Register of Historic Places."
1,http://dbpedia.org/resource/Carl_Koch_(architect),"Carl Koch (May 11, 1912 – 3 July 3, 1998) was a noted American architect. He was most associated with the design of prefabricated homes and development of the Techcrete building system."
2,http://dbpedia.org/resource/Edward_Brickell_White,"Edward Brickell White, auch bekannt als E. B. White (* 29. Januar 1806 in St. John, South Carolina; † 10. Mai 1882 in New York), war ein US-amerikanischer Architekt, der vor allem durch seine neugotische Architektur und die Verwendung römischer und klassischer griechischer Designentwürfe bekannt wurde."
3,http://dbpedia.org/resource/Edward_Brickell_White,"Edward Brickell White (January 29, 1806 – May 10, 1882), also known as E. B. White, was an American architect. He was known for his Gothic Revival architecture and his use of Roman and Greek designs."
4,http://dbpedia.org/resource/Samuel_B._Reed,"Samuel Burrage Reed was an American architect of Corona, New York and Woodcliff Lake, New Jersey. He was active in mid-to-late and early twentieth-century America, particularly in New York State, New York City, and Connecticut. Born in Meriden, Connecticut, he was first trained as a carpenter before becoming an architect. He is notable for designing several mansions, public and ecclesiastical ..."
5,http://dbpedia.org/resource/Alexander_C._Eschweiler,"Alexander Chadbourne Eschweiler (August 10, 1865 – June 12, 1940) was an American architect with a practice in Milwaukee, Wisconsin. He designed both residences and commercial structures. His eye-catching Japonist pagoda design for filling stations for Wadham's Oil and Grease Company of Milwaukee were repeated over a hundred times, though only a very few survive. His substantial turn-of-the-20..."
6,http://dbpedia.org/resource/Doug_Suisman,"Doug Suisman (born 14 March 1955) is an award-winning American urban designer and architect. Suisman founded the Los Angeles-based firm, Suisman Urban Design, in 1990 and has since worked on projects in a variety of cities including Los Angeles, Ramallah, and Vancouver, among others. His work emphasizes sustainable development, public transportation, communal spaces and structures, and walkabl..."
7,http://dbpedia.org/resource/D._Everett_Waid,"Dan Everett Waid (1864–1939) was a prominent 20th century architect operating primarily in Illinois and New York. As chief architect for the Metropolitan Life Insurance Company (New York City), he and his partner designed the Home Office Building at 11 Madison Avenue along with dozens of other commercial, religious, residential and academic structures. He was appointed architect for the Board ..."
8,http://dbpedia.org/resource/Gladys_J._Miller,"Gladys Jacqueline Good ""Hap"" Miller (December 9, 1926- September, 1993) was an American architect active in Terre Haute, Indiana from 1950 to 1979, primarily specializing in residential architecture."
9,http://dbpedia.org/resource/Frederick_Carles_Merry,"Frederick Carles Merry, AIA, (d. 1900, New York City) was an American architect active in late-nineteenth-century New York City. Merry was born in England and emigrated to the United States as a child. He worked for an architectural firm in Philadelphia before becoming principal in the New York City office of Henry Hobson Richardson's architectural firm. He opened his own practice at some poin..."


## Task 

# Project Data collection

Multi-label sentiment analysis classification 


**Мета** <br>
- Understanding the problem <br>
Для себе сформована мета - це, маючи,  суб’єктивне написане живою мовою висловлювання, визначити наявність n кількості емоцій, на даному етапі не вдаючись в деталізацію стосовно об’єкту (не aspect-based).

**Ціль:** multi-domain sentiment analysis (that’s why Twitter about a variety of topics unlike other sites which are tailored to a specific topic.)


Як збирати дані? <br>
Окрім знайдених датасетів, можна зібрати дані за допомогою emojis(смайлів). <br>
Idea <br>
- Emojies -> labeled data <br>
**Datasets** <br>
1. Twemoji Dataset - датасет для моделі, де текст приймається на вхід, а - смайл пронозується. Звдіси можна витягти за певними смайлами емоції(мітки).
[https://uvaauas.figshare.com/articles/Twemoji_Dataset/5822100]
3. тільки негативні https://data.world/crowdflower/mcdonalds-review-sentiment
4. Датасет з 18 класами емоцій, у моєму первинному датасеті 11 класів, можна спробувати знайти відповідності https://data.world/crowdflower/primary-emotions-of-statements
5. Датасет для прогнозування смайлу плескання руками (можливо, використати) https://www.kaggle.com/rtatman/clap-emoji-in-tweets

Також, крім цього як Data Sources можна самостійно назбирати дані з Reddit/Twitter, з наявними смайлами як мітками. <br>
Для цього статті: <br>
**Як обробляти дані витягнені з твітеру ** <br>
Стаття [https://cs.stanford.edu/people/alecmgo/papers/TwitterDistantSupervision09.pdf] <br>
Twitter API example https://www.linkedin.com/pulse/social-machine-learning-h2o-twitter-python-marios-michailidis <br>
<br>
Цікаво: <br>
- emoticons самі по собі з вибірки видаляються
- твіти з позитивними та негативними емоціми одночасно видаляти з вибірки?
- ретвіти видаляти


**Лише для особистого перегляду**
1. Знайти статистику у твітері за емоціями (структуровані дані)
http://emojitracker.com (використанні емоцій)

2. Tag clouds.  Given a list of positive and negative tweets, what are the most meaningful words to put in a tag cloud?

3. Emojies sentiment ranking http://kt.ijs.si/data/Emoji_sentiment_ranking/

Розмір корпусу 
Small - 10k - 10m tokens
Big - 1G